In [4]:
! pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 452 kB 20.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 98.2 MB/s eta 0:00:01
     |████████████████████████████████| 213 kB 104.4 MB/s eta 0:00:01
     |████████████████████████████████| 110 kB 99.2 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 106.6 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 97.0 MB/s eta 0:00:01
     |████████████████████████████████| 262 kB 103.6 MB/s eta 0:00:01
     |████████████████████████████████| 161 kB 86.0 MB/s eta 0:00:01


In [6]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 81 kB 22.9 MB/s eta 0:00:01


In [2]:
import evaluate
import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset

In [3]:
data = pd.read_csv("data.csv")
train, val = train_test_split(data, test_size=0.2, shuffle=True, random_state=32)
train = Dataset.from_pandas(train)
val = Dataset.from_pandas(val)

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


train = train.map(tokenize_function, batched=True)
val = val.map(tokenize_function, batched=True)

100%|██████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.42ba/s]


In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("./test_trainer/checkpoint-1000/", num_labels=2).cuda()

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [7]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    compute_metrics=compute_metrics,
)

In [50]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 11338
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2127
  Number of trainable parameters = 108311810
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.252100,0.204171,0.934039
2,0.187400,0.224383,0.924868
3,0.081100,0.280093,0.935802


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2835
  Batch size = 16
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will inste

TrainOutput(global_step=2127, training_loss=0.1580676055032993, metrics={'train_runtime': 334.5804, 'train_samples_per_second': 101.662, 'train_steps_per_second': 6.357, 'total_flos': 8949459437015040.0, 'train_loss': 0.1580676055032993, 'epoch': 3.0})

In [51]:
def tf_lower_and_split_punct(text):
  # Split accented characters.
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)
  
  #text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [54]:
text = [data['text'][i] for i in range(len(data['text']))]
df_new = pd.DataFrame()
df_new['text'] = text
df_new['label'] = data['label']
train, val = train_test_split(df_new, test_size=0.2, shuffle=True, random_state=32)
train = Dataset.from_pandas(train)
val = Dataset.from_pandas(val)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


train = train.map(tokenize_function, batched=True)
val = val.map(tokenize_function, batched=True)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

import numpy as np
import evaluate

metric = evaluate.load("accuracy")

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", learning_rate=5e-06)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    compute_metrics=compute_metrics,
)

trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
loading file tokenize

Epoch,Training Loss,Validation Loss,Accuracy
1,0.288500,0.209574,0.933333
2,0.187600,0.220869,0.935450
3,0.155600,0.227803,0.934744


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2835
  Batch size = 16
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will inste

TrainOutput(global_step=2127, training_loss=0.19790714516794397, metrics={'train_runtime': 511.5902, 'train_samples_per_second': 66.487, 'train_steps_per_second': 4.158, 'total_flos': 8949459437015040.0, 'train_loss': 0.19790714516794397, 'epoch': 3.0})

In [12]:
trainer.predict(val)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2835
  Batch size = 16
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


PredictionOutput(predictions=array([[ 1.8414098, -3.166564 ],
       [ 2.0747056, -2.999138 ],
       [ 1.7625471, -2.7578676],
       ...,
       [ 2.3990955, -3.222688 ],
       [ 1.5202812, -2.7088852],
       [-1.6973903,  1.8626778]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 1]), metrics={'test_loss': 0.2135286033153534, 'test_accuracy': 0.9340388007054674, 'test_runtime': 43.2031, 'test_samples_per_second': 65.62, 'test_steps_per_second': 4.12})